In [43]:
import zipfile

import folium
import geopandas as gpd
from lxml import etree
import pandas as pd
import requests

In [36]:
# メンテナンスエリア
url = 'https://network.mobile.rakuten.co.jp/assets/json/maintenance-short.json'
res = requests.get(url)
data = res.json()
df0 = pd.json_normalize(data)
df1 = df0.query('都道府県 == "長野"').reset_index(drop=True)
df2 = pd.concat([df1, df1['対象地域'].str.split('の一部地域', expand=True)], axis=1)
df2.rename(columns={0: '対象地域_split'}, inplace=True)
df2.drop(columns=1, inplace=True)
df2

,都道府県,短期メンテナンス工事,作業開始時間,作業終了時間,お客様への影響,対象地域,対象地域_split
0,長野,,2022年6月16日 1:00,2022年6月16日 5:00,作業時間内で30分未満の通信断が発生し、音声通話およびデータ通信が、ご利用いただけない場合が...,長野県松本市大字和田字東沖の一部地域,長野県松本市大字和田字東沖
1,長野,,2022年6月16日 1:00,2022年6月16日 5:00,作業時間内で30分未満の通信断が発生し、音声通話およびデータ通信が、ご利用いただけない場合が...,長野県松本市大字新村字道端の一部地域,長野県松本市大字新村字道端
2,長野,,2022年6月16日 1:00,2022年6月16日 5:00,作業時間内で30分未満の通信断が発生し、音声通話およびデータ通信が、ご利用いただけない場合が...,長野県松本市大字神林字南水寺の一部地域,長野県松本市大字神林字南水寺
3,長野,,2022年6月16日 1:00,2022年6月16日 5:00,作業時間内で30分未満の通信断が発生し、音声通話およびデータ通信が、ご利用いただけない場合が...,長野県松本市大字島内字寺村の一部地域,長野県松本市大字島内字寺村
4,長野,,2022年6月16日 1:00,2022年6月16日 5:00,作業時間内で30分未満の通信断が発生し、音声通話およびデータ通信が、ご利用いただけない場合が...,長野県松本市大字和田字下野尻の一部地域,長野県松本市大字和田字下野尻
...,...,...,...,...,...,...,...
198,長野,,2022年6月16日 1:00,2022年6月16日 5:00,作業時間内で30分未満の通信断が発生し、音声通話およびデータ通信が、ご利用いただけない場合が...,長野県佐久市甲字中原前の一部地域,長野県佐久市甲字中原前
199,長野,,2022年6月16日 1:00,2022年6月16日 5:00,作業時間内で30分未満の通信断が発生し、音声通話およびデータ通信が、ご利用いただけない場合が...,長野県佐久市瀬戸字橋詰の一部地域,長野県佐久市瀬戸字橋詰
200,長野,,2022年6月16日 1:00,2022年6月16日 5:00,作業時間内で30分未満の通信断が発生し、音声通話およびデータ通信が、ご利用いただけない場合が...,長野県佐久市三塚字北井戸の一部地域,長野県佐久市三塚字北井戸
201,長野,,2022年6月16日 1:00,2022年6月16日 5:00,作業時間内で30分未満の通信断が発生し、音声通話およびデータ通信が、ご利用いただけない場合が...,長野県佐久市新子田字柳反りの一部地域,長野県佐久市新子田字柳反り


In [37]:
list(df0['都道府県'].unique())

['北海道', '新潟', '長野', '岡山', '広島', '山口', '富山']

In [38]:
gdf0 = gpd.read_file('A002005212015DDSWC20.zip!h27ka20.shp')
gdf0['S_NAME_2'] = gdf0['S_NAME'].replace('\d丁目', '', regex=True)
gdf0['addr'] = gdf0['PREF_NAME'] + gdf0['CITY_NAME'] + gdf0['S_NAME_2']
gdf1 = gdf0[gdf0['addr'].isin(df2['対象地域_split'])].reset_index(drop=True)
gdf1

,KEY_CODE,PREF,CITY,S_AREA,PREF_NAME,CITY_NAME,S_NAME,KIGO_E,HCODE,AREA,...,MOJI,KBSUM,JINKO,SETAI,X_CODE,Y_CODE,KCODE1,geometry,S_NAME_2,addr
0,20201006001,20,201,006001,長野県,長野市,上松１丁目,None,8101,1.940868e+05,...,上松１丁目,19,1203,554,138.19997,36.66826,0060-01,"POLYGON ((138.19943 36.66659, 138.19939 36.666...",上松,長野県長野市上松
1,20201006002,20,201,006002,長野県,長野市,上松２丁目,None,8101,3.785289e+05,...,上松２丁目,33,1657,754,138.19355,36.66776,0060-02,"POLYGON ((138.19590 36.66522, 138.19586 36.665...",上松,長野県長野市上松
2,20201006003,20,201,006003,長野県,長野市,上松３丁目,None,8101,5.117088e+05,...,上松３丁目,41,1226,507,138.19659,36.67367,0060-03,"POLYGON ((138.19287 36.66972, 138.19257 36.669...",上松,長野県長野市上松
3,20201006004,20,201,006004,長野県,長野市,上松４丁目,None,8101,3.592507e+05,...,上松４丁目,44,2212,907,138.20199,36.67586,0060-04,"POLYGON ((138.20334 36.67071, 138.20256 36.670...",上松,長野県長野市上松
4,20201006005,20,201,006005,長野県,長野市,上松５丁目,None,8101,5.029666e+05,...,上松５丁目,20,989,357,138.19637,36.68107,0060-05,"POLYGON ((138.19619 36.67744, 138.19604 36.677...",上松,長野県長野市上松
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101,20218029004,20,218,029004,長野県,千曲市,上山田温泉４丁目,None,8101,1.437653e+05,...,上山田温泉４丁目,25,373,158,138.14650,36.47295,0290-04,"POLYGON ((138.14877 36.47152, 138.14879 36.471...",上山田温泉,長野県千曲市上山田温泉
102,202200050,20,220,005022,長野県,安曇野市,豊科光,None,8101,3.936964e+06,...,豊科光,12,1144,370,137.92118,36.32470,0050-22,"POLYGON ((137.93514 36.31319, 137.93458 36.313...",豊科光,長野県安曇野市豊科光
103,20220008026,20,220,008026,長野県,安曇野市,穂高有明,None,8101,5.578366e+07,...,穂高有明,2,39,39,137.76025,36.38188,0080-26,"POLYGON ((137.81533 36.39225, 137.81483 36.390...",穂高有明,長野県安曇野市穂高有明
104,203210090,20,321,009001,長野県,軽井沢町,中軽井沢,None,8101,2.014821e+05,...,中軽井沢,8,550,231,138.59117,36.35281,0090-01,"POLYGON ((138.59471 36.34888, 138.59453 36.348...",中軽井沢,長野県軽井沢町中軽井沢


In [39]:
def func(v):

    start_time = df2.query('対象地域_split == @v')['作業開始時間'].iloc[-1]
    end_time = df2.query('対象地域_split == @v')['作業終了時間'].iloc[-1]
    area = df2.query('対象地域_split == @v')['対象地域'].iloc[-1]

    return pd.Series([start_time, end_time, area])


In [40]:
gdf1[['作業開始時間', '作業終了時間', '対象地域']] = gdf1['addr'].apply(func)
gdf2 = gdf1[['作業開始時間', '作業終了時間', '対象地域', 'geometry']]
gdf2

,作業開始時間,作業終了時間,対象地域,geometry
0,2022年6月16日 1:00,2022年6月16日 5:00,長野県長野市上松の一部地域,"POLYGON ((138.19943 36.66659, 138.19939 36.666..."
1,2022年6月16日 1:00,2022年6月16日 5:00,長野県長野市上松の一部地域,"POLYGON ((138.19590 36.66522, 138.19586 36.665..."
2,2022年6月16日 1:00,2022年6月16日 5:00,長野県長野市上松の一部地域,"POLYGON ((138.19287 36.66972, 138.19257 36.669..."
3,2022年6月16日 1:00,2022年6月16日 5:00,長野県長野市上松の一部地域,"POLYGON ((138.20334 36.67071, 138.20256 36.670..."
4,2022年6月16日 1:00,2022年6月16日 5:00,長野県長野市上松の一部地域,"POLYGON ((138.19619 36.67744, 138.19604 36.677..."
...,...,...,...,...
101,2022年6月16日 1:00,2022年6月16日 5:00,長野県千曲市上山田温泉の一部地域,"POLYGON ((138.14877 36.47152, 138.14879 36.471..."
102,2022年6月16日 1:00,2022年6月16日 5:00,長野県安曇野市豊科光の一部地域,"POLYGON ((137.93514 36.31319, 137.93458 36.313..."
103,2022年6月16日 1:00,2022年6月16日 5:00,長野県安曇野市穂高有明の一部地域,"POLYGON ((137.81533 36.39225, 137.81483 36.390..."
104,2022年6月16日 1:00,2022年6月16日 5:00,長野県軽井沢町中軽井沢の一部地域,"POLYGON ((138.59471 36.34888, 138.59453 36.348..."


In [54]:
with zipfile.ZipFile("rakuten.kmz") as existing_zip:
    existing_zip.extractall("kmz")

ns = {"kml": "http://www.opengis.net/kml/2.2"}
tree = etree.parse("./kmz/doc.kml")

data = []

for i in tree.xpath("//kml:Placemark", namespaces=ns):

    d = {}

    d["名前"] = "".join(i.xpath("./kml:name/text()", namespaces=ns)).strip()
    d["geometry"] = "".join(
        i.xpath("./kml:Point/kml:coordinates/text()", namespaces=ns)
    ).strip()

    for j in i.xpath("./kml:ExtendedData/kml:Data", namespaces=ns):

        k = j.attrib["name"].strip()
        v = "".join(j.xpath("./kml:value/text()", namespaces=ns)).strip()

        d[k] = v

    data.append(d)

df_map_1 = pd.DataFrame(data)

df_map_1

df_map_1[["経度", "緯度", "標高"]] = df_map_1["geometry"].str.split(",", expand=True)

# 文字から数値に変換
df_map_1["経度"] = pd.to_numeric(df_map_1["経度"])
df_map_1["経度"] = pd.to_numeric(df_map_1["経度"])

# df_map.columns
# df_map['開局状態'].unique() # ['開局', '未開局', nan]
df_map_2 = df_map_1.query('開局状態 == "開局" | 開局状態 == "未開局"')

df_map_3 = df_map_2[['eNB-LCID', '住所', '市区町村', '開局状態', '経度', '緯度']].copy()

df_map_3

,eNB-LCID,住所,市区町村,開局状態,経度,緯度
0,"212995-1,2,3",長野県塩尻市広丘郷原１７６２−２８８,塩尻市,開局,137.932062,36.119826
1,"212995-10,11,12",長野県塩尻市大門八番町９−１,塩尻市,開局,137.948550,36.11327
2,"212995-4,5,6",長野県塩尻市大門９５−１１,塩尻市,開局,137.946973,36.123974
3,"212995-7,8,9",長野県塩尻市桔梗ケ原７１−３６９,塩尻市,開局,137.942787,36.11194
4,"212997-1,2,3",長野県松本市内田４５−１,松本市,開局,137.983550,36.17247
...,...,...,...,...,...,...
193,213198-3,長野県大町市平,大町市,開局,137.830120,36.51402
194,"213198-7,8",長野県大町市平,大町市,開局,137.812290,36.52665
195,213200-10,長野県安曇野市穂高有明,安曇野市,開局,137.861770,36.35479
196,"213201-10,11,12",長野県北安曇郡松川村,北安曇郡松川村,開局,137.867930,36.40412


In [72]:
# 行政区域データ
gdf_area = gpd.read_file('N03-20210101_20_GML.zip!N03-20210101_20_GML/N03-21_20_210101.shp')

gdf_chusin = gdf_area[gdf_area['N03_004'].isin(['松本市', '塩尻市', '安曇野市', '大町市', '麻績村', '生坂村', '山形村', '朝日村', '筑北村', '池田町', '松川村', '白馬村', '小谷村', '上松町', '南木曽町', '木祖村', '王滝村','大桑村', '木曽町'])]
gdf_chusin

,N03_001,N03_002,N03_003,N03_004,N03_007,geometry
1,長野県,None,None,松本市,20202,"POLYGON ((138.05548 36.37721, 138.05549 36.377..."
11,長野県,None,None,大町市,20212,"POLYGON ((137.76860 36.65856, 137.76866 36.658..."
14,長野県,None,None,塩尻市,20215,"POLYGON ((137.96450 36.17149, 137.96475 36.171..."
19,長野県,None,None,安曇野市,20220,"POLYGON ((137.72955 36.42532, 137.73017 36.425..."
54,長野県,None,木曽郡,上松町,20422,"POLYGON ((137.72809 35.82215, 137.72833 35.822..."
55,長野県,None,木曽郡,南木曽町,20423,"POLYGON ((137.53581 35.69008, 137.53597 35.690..."
56,長野県,None,木曽郡,木祖村,20425,"POLYGON ((137.77616 36.07500, 137.77620 36.074..."
57,長野県,None,木曽郡,王滝村,20429,"POLYGON ((137.43761 35.90505, 137.43766 35.903..."
58,長野県,None,木曽郡,大桑村,20430,"POLYGON ((137.81205 35.75964, 137.81192 35.759..."
59,長野県,None,木曽郡,木曽町,20432,"POLYGON ((137.63561 36.02590, 137.63574 36.025..."


In [80]:
# ベースマップ
map = folium.Map(
    tiles = None,
    location = [36.651278, 138.180944],
    zoom_start = 11,
    control_scale = True
)

# Googleマップ標準
folium.raster_layers.TileLayer(
    'https://{s}.google.com/vt/lyrs=m&x={x}&y={y}&z={z}',
    subdomains = ['mt0','mt1','mt2','mt3'],
    name = "Google Map",
    attr = "<a href='https://developers.google.com/maps/documentation' target='_blank'>Google Map</a>"
).add_to(map)

cell_group = folium.FeatureGroup(name="基地局").add_to(map)

for i, r in df_map_3.iterrows():

    # popup
    # eNB-LCID, 住所, 市区町村, 開局状態	
    html = f'''
    <p>eNB-LCID: {r['eNB-LCID']}</p>
    <p>住所: {r['住所']}</p>
    <p>市区町村: {r['市区町村']}</p>
    <p>開局状態: {r['開局状態']}</p>
    '''

    cell_group.add_child(
        folium.Marker(
            location = [ r["緯度"], r["経度"] ],
            popup = folium.Popup(html=html, max_width=300),
            # icon=folium.Icon(color = icon)
            icon=folium.Icon(color = 'pink' if r['開局状態'] == '開局' else 'black')
        )
    )

# 楽天モバイルエリア4Gマップレイヤー(予定1)
folium.raster_layers.TileLayer(
    name="(4G)楽天モバイルエリア(予定1)",
    tiles='https://gateway-api.global.rakuten.com/dsd/geoserver/4g2m/mno_coverage_map/gwc/service/gmaps?LAYERS=mno_coverage_map:all_map&FORMAT=image/png&TRANSPARENT=TRUE&x={x}&y={y}&zoom={z}',
    fmt='image/png',
    attr="<a href='https://network.mobile.rakuten.co.jp/'>楽天モバイル</a>",
    tms=False,
    overlay=True,
    control=True,
    opacity=1.0,
    show=False,
).add_to(map)

# 楽天モバイルエリア4Gマップレイヤー(予定2)
folium.raster_layers.TileLayer(
    name="(4G)楽天モバイルエリア(予定2)",
    tiles='https://gateway-api.global.rakuten.com/dsd/geoserver/4g4m/mno_coverage_map/gwc/service/gmaps?LAYERS=mno_coverage_map:all_map&FORMAT=image/png&TRANSPARENT=TRUE&x={x}&y={y}&zoom={z}',
    fmt='image/png',
    attr="<a href='https://network.mobile.rakuten.co.jp/'>楽天モバイル</a>",
    tms=False,
    overlay=True,
    control=True,
    opacity=1.0,
    show=False,
).add_to(map)

# 楽天モバイルエリア5Gマップレイヤー
folium.raster_layers.TileLayer(
    name="(5G)楽天モバイルエリア",
    tiles='https://gateway-api.global.rakuten.com/dsd/geoserver/5g/mno_coverage_map/gwc/service/gmaps?LAYERS=mno_coverage_map:all_map&FORMAT=image/png&TRANSPARENT=TRUE&x={x}&y={y}&zoom={z}',
    fmt='image/png',
    attr="<a href='https://network.mobile.rakuten.co.jp/'>楽天モバイル</a>",
    tms=False,
    overlay=True,
    control=True,
    opacity=1.0,
    show=False,
).add_to(map)

folium.GeoJson(
    data = gdf1.to_json(),
    style_function = lambda x:{
        "fillColor": 'red',
        'fillOpacity': 0.7,
        "color": "black",
        "weight": 1
    },
    name = '短期メンテナンス工事エリア',
    show = True,
    tooltip = folium.features.GeoJsonTooltip(
        fields=['対象地域', '作業開始時間', '作業終了時間', ],
        aliases=['対象地域', '作業開始時間', '作業終了時間',  ]
    )
).add_to(map)

# 中信エリア
folium.features.GeoJson(
    data = gdf_chusin.to_json(),
    style_function = lambda x:{
        'fillColor': '#000000',
        'fillOpacity': 0,
        'color' : '#FF0000',
        'weight': 1.5
    },
    name = "行政区域(出典:国土交通省)",
    show = True,
    popup = folium.features.GeoJsonPopup(
        fields = ['N03_003', 'N03_004'],
        aliases = ['郡・政令都市名', '市区町村名']
    ),
).add_to(map)

# レイヤーコントロール
folium.LayerControl().add_to(map)

map
map.save('map.html')